In [1]:
import json

with open("authors.json", "r") as json_file:
    authors = json.load(json_file)

print(authors)

{'Достоевский Ф. М.': 9150, 'Роллинс Дж.': 59396, 'Фицджеральд Ф. С.': 28727, 'Глуховский Д. А.': 53427, 'Стругацкий А. Н.': 26268, 'Лукьяненко С. В.': 16626, 'Фрай М.': 28927, 'Хантер Э.': 37969, 'Роулинг Дж. К.': 104832}


In [31]:
from bs4 import BeautifulSoup
import requests

def get_all_books_by_author_id(author_id):
    base_url = "https://www.moscowbooks.ru"
    ids = []
    page_number = 1
    while True:
        page = requests.get(base_url + 
                            "/catalog/author/{0}/?d_acc=any&page={1}".format(author_id, page_number))
        soup = BeautifulSoup(page.text, 'html.parser')
        for tag in soup.find_all("a", {'data-producttype':'0'}):
            ids.append(tag.attrs['data-productid'])
        if soup.find("a", {'title':'следующая страница'}):
            page_number += 1
        else:
            break
    return ids

In [106]:
import re


def extract_book_info(book_id):
    base_url = "https://www.moscowbooks.ru"
    card = {"Код товара": book_id}
    page = requests.get(base_url + "/book/{}".format(book_id))
    soup = BeautifulSoup(page.text, 'html.parser')
    card["Автор"] = re.sub("\[|\]|'|\n", "", soup.find(
        'div', {'class': 'page-header__author'}).text).replace(',', ', ')
    card["Название"] = soup.find('div', {'itemprop': "name"}).text
    card["Обложка"] = base_url + \
        soup.find('a', {'itemprop': "image"}).attrs['href']
    card['Рейтинг'] = soup.find('div', {'class': "book__rating"}).find(
        'div', {'class': "book___rating-stars"}).attrs['data-rate']
    stickers = soup.find('div', {'class': 'book__stickers'})
    if stickers != None:
        stickers = [tag.findChild() for tag in
                    stickers.findAll('div', {'class': 'stickers__item'})]
        card['Стикеры'] = ', '.join(list(map(lambda x: x.attrs['title']
                                   if x.has_attr('title') else x.get_text(), stickers)))
    else:
        card['Стикеры'] = ' '
    card['Наличие'] = len(
        soup.find('div', {'class': "book__shop-details"}).contents) > 0
    price = soup.find('div', {'class': 'book__price'})
    card['Цена'] = re.sub(r'\D', '', price.text) if price else None
    table = soup.find('div', {'class': 'book__details-list'})
    for tag, value in zip(table.findAll('dt', {'class': 'book__details-name'}), table.findAll('dt', {'class': 'book__details-value'})):
        name = tag.text.strip('\r\n ')
        if name == 'Издательство:' or name == 'Производитель:':
            name = 'Издательство\Производитель:'
        elif name == 'Год издания:' or name == 'Год производства:':
            name = 'Год издания\производства:'
        elif name == 'Место издания:' or name == 'Место производства:':
            name = 'Место издания\производства:'
        card[name] = value.text.strip('\r\n ')
    card['Описание'] = soup.find('div', {'class': 'book__description collapsed js-book-description'}
                                 ).findAll(text=True, recursive=False)[1].strip('\r\n ')
    return card

In [107]:
import pandas as pd
from itertools import chain

book_ids = list(chain(*list(map(get_all_books_by_author_id, list(authors.values())))))
result = list(map(extract_book_info, book_ids))
df = pd.DataFrame(result)
df.sort_values(by=['Код товара'], inplace=True)

with open('data/hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [108]:
res = pd.read_csv('data/hw_3.csv')

In [109]:
res.head()

,Код товара,Автор,Название,Обложка,Рейтинг,Стикеры,Наличие,Цена,Издательство\Производитель:,Год издания\производства:,...,Описание,Тиснение:,Футляр:,Бумага:,Артикул:,Иллюстрации:,Возраст:,Язык оригинала:,Перевод:,Редактор/составитель:
0,1000044,Фицджеральд Ф. С.,Великий Гэтсби = The Great Gatsby. Уникальная ...,https://www.moscowbooks.ru/image/book/664/w259...,0,NaN,True,224,АСТ,2019,...,Лучший способ учить иностранный язык — это чит...,NaN,NaN,NaN,ASE000000000842319,NaN,NaN,NaN,NaN,NaN
1,1000435,Достоевский Ф. М.,Чужая жена и муж под кроватью,https://www.moscowbooks.ru/image/book/664/w259...,0,NaN,True,144,АСТ,2019,...,Очень необычный сборник. Очень непривычный Дос...,NaN,NaN,NaN,ASE000000000843846,NaN,NaN,NaN,NaN,NaN
2,1000624,"Стругацкий А. Н., Стругацкий Б. Н.",Полное собрание сочинений. В 33 томах. Том 16....,https://www.moscowbooks.ru/image/book/664/w259...,0,NaN,True,3000,Издательство Сидорович,2019,...,Полное собрание сочинений Аркадия и Бориса Стр...,NaN,NaN,NaN,978-5-905909-26-9,NaN,NaN,NaN,NaN,NaN
3,1000625,"Стругацкий А. Н., Стругацкий Б. Н.",Полное собрание сочинений. В 33 томах. Том 17....,https://www.moscowbooks.ru/image/book/664/w259...,0,NaN,True,3000,Издательство Сидорович,2019,...,Полное собрание сочинений Аркадия и Бориса Стр...,NaN,NaN,NaN,978-5-905909-27-6,NaN,NaN,NaN,NaN,NaN
4,1002955,Фицджеральд Ф. С.,Ночь нежна,https://www.moscowbooks.ru/image/book/666/w259...,0,NaN,True,160,Эксмо,2019,...,"""Ночь нежна"" — удивительно тонкий и глубоко пс...",NaN,NaN,NaN,ITD000000000950861,NaN,NaN,английский,Ильин С.,NaN
